# EarlyStruct – Design Explorer (Floors)
Carbon-first feasibility & comparison tool.

**How to use**
1) Set **Data dir** to the folder containing your CSVs.
2) (Optional) Provide a **Control file** path. If present, it takes precedence for project details and SPANS unless it contains `USE_CSV = Y`.
3) (Optional) Enter **Spans** like `9, 10.5` (meters) or `28ft, 32ft`. Notebook spans override control-file SPANS.
4) Click **Run Evaluation**. If no spans are provided (and no grid_options.csv / ideal spacing), the tool runs a 1-ft sweep (18–45 ft).


In [8]:
import sys, os
import pandas as pd
import matplotlib.pyplot as plt
pkg_root = os.path.abspath(os.path.join('..'))
if pkg_root not in sys.path:
    sys.path.insert(0, pkg_root)
from earlystruct.cli import evaluate
print('earlystruct package ready')


earlystruct package ready


In [ ]:
from ipywidgets import Text, Button, VBox, HTML
data_dir = Text(value='/Users/benjaminsalop/Desktop/Oxford/Research/edca/csvs', description='Data dir:', layout={'width':'70%'})
control_file = Text(value='/Users/benjaminsalop/Desktop/Oxford/Research/edca/control_files/control_file.txt', description='Control:', layout={'width':'70%'})
spans = Text(value='6', description='Spans:')
run_btn = Button(description='Run Evaluation', button_style='primary')
status = HTML(value='')
VBox([data_dir, control_file, spans, run_btn, status])

In [10]:
from ipywidgets import Output
from IPython.display import display, clear_output
import pandas as pd

out = out if 'out' in globals() else Output()  # if you haven't already created it in the UI cell
SHOW_COLS = ["span_m","system_id","system_name","category","type","manufacturer",
             "depth_m","carbon_per_m2","cost_per_m2","edge_canti_x_m","edge_canti_y_m","reason","on_pareto"]

def on_run(_):
    with out:
        clear_output(wait=True)
        status.value = "<b>Running…</b>"
        try:
            cf = control_file.value.strip() or None
            df, ranked, pareto, _ = evaluate(data_dir.value, spans.value or None, None, cf)
            feas = df[df["feasible"]].copy()
            if "on_pareto" in pareto.columns:
                feas = feas.merge(pareto[["system_id","span_m","on_pareto"]], how="left", on=["system_id","span_m"])
            status.value = f"<b>Done.</b> All: {len(df)} | Feasible: {len(feas)}"
            cols = [c for c in SHOW_COLS if c in feas.columns]
            display(feas[cols].sort_values(["carbon_per_m2","cost_per_m2"], na_position="last").reset_index(drop=True))
        except Exception as e:
            status.value = f'<span style="color:red">Error: {e}</span>'

run_btn.on_click(on_run)
out


Output()

## Pareto Chart (Carbon vs Cost)
Plot uses the latest run's feasible set.

In [5]:
def plot_pareto():
    if 'ranked' not in results or results['ranked'] is None or results['ranked'].empty:
        print('Run evaluation first.'); return
    df = results['ranked']
    pareto = results.get('pareto', pd.DataFrame())
    plt.figure()
    plt.scatter(df['carbon_per_m2'], df['cost_per_m2'])
    if not pareto.empty:
        p = pareto[pareto['on_pareto']]
        if not p.empty:
            plt.scatter(p['carbon_per_m2'], p['cost_per_m2'])
    plt.xlabel('Carbon (kgCO2e/m²)')
    plt.ylabel('Cost (per m²)')
    plt.title('Pareto: Carbon vs Cost (Feasible)')
    plt.show()
plot_pareto()

NameError: name 'results' is not defined